<a href="https://colab.research.google.com/gist/maclandrol/aa48a4bdba09ddae1f5158d715bc5671/10_TorchXRayVision_Dataset_Personnalise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Enseignant:** Emmanuel Noutahi, PhD

# Tutorial 6 : Intégration de Datasets Personnalisés
# Tutorial 6: Custom Dataset Integration

---

## 📚 Contexte Médical / Medical Context

### Pour les Étudiants en Médecine / For Medical Students
L'**intégration de données personnalisées** est essentielle pour :
- **Recherche médicale** : Analyser vos propres données cliniques
- **Études spécialisées** : Focus sur pathologies spécifiques
- **Validation locale** : Adapter l'IA à votre population
- **Projets étudiants** : Mener des recherches originales

### Pour les Praticiens / For Practitioners
- **Chirurgiens** : Validation sur cas chirurgicaux spécifiques
- **Médecins Généralistes** : Adaptation aux spécificités locales
- **Enseignants** : Création de cas pédagogiques personnalisés

---

## 🎯 Objectifs d'Apprentissage / Learning Objectives

À la fin de ce tutoriel, vous serez capable de :
1. **Charger facilement** vos propres images radiologiques
2. **Organiser** un dataset personnalisé pour l'analyse
3. **Appliquer** les modèles TorchXRayVision sur vos données
4. **Analyser en lot** plusieurs images simultanément
5. **Générer** des rapports comparatifs pour votre dataset
6. **Exporter** les résultats pour usage clinique ou recherche

---

## 💡 Fonctionnalités Colab Faciles / Easy Colab Features

Ce tutoriel est conçu pour **Google Colab** avec :
- **Glisser-déposer** d'images multiples
- **Organisation automatique** des données
- **Traitement par lot** simplifié
- **Visualisations interactives**
- **Export automatique** des résultats

---

## 📋 Prérequis / Prerequisites

Ce tutoriel fait suite aux **Tutoriels 1-4**. Vous devriez maîtriser :
- Utilisation de base de TorchXRayVision
- Classification et détection de pathologies
- Interprétation des résultats

---

## 🔧 Installation et Configuration / Setup

In [ ]:
# Installation des bibliothèques nécessaires / Install required libraries
!pip install torchxrayvision
!pip install torch torchvision
!pip install matplotlib seaborn
!pip install numpy pandas
!pip install scikit-image opencv-python
!pip install tqdm  # Barre de progression
!pip install zipfile36  # Pour gérer les archives

print("✅ Installation terminée / Installation completed")

In [ ]:
# Import des bibliothèques / Import libraries
import torch
import torch.nn as nn
import torchxrayvision as xrv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from PIL import Image
import cv2
import os
import glob
import zipfile
import io
import json
from google.colab import files
from tqdm import tqdm
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage / Display configuration
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print("📚 Bibliothèques importées avec succès / Libraries imported successfully")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🏥 TorchXRayVision version: {xrv.__version__}")

# Vérification du GPU / GPU check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"💻 Device utilisé / Device used: {device}")

# Créer dossier de travail pour les données
os.makedirs('custom_dataset', exist_ok=True)
os.makedirs('results', exist_ok=True)
print("📁 Dossiers de travail créés / Working directories created")

## 🤖 Chargement des Modèles / Loading Models

In [ ]:
# Chargement des modèles TorchXRayVision / Load TorchXRayVision models
print("🔄 Chargement des modèles pour analyse de dataset personnalisé...")
print("🔄 Loading models for custom dataset analysis...")

models = {}
model_info = {}

# 1. Modèle principal pour classification
try:
    models['densenet'] = xrv.models.DenseNet(weights="densenet121-res224-all")
    models['densenet'].to(device)
    models['densenet'].eval()
    model_info['densenet'] = {
        'name': 'DenseNet121-All',
        'pathologies': models['densenet'].pathologies,
        'description': 'Modèle général pour toutes pathologies'
    }
    print("✅ DenseNet121-All chargé / loaded")
except Exception as e:
    print(f"❌ Erreur DenseNet: {e}")

# 2. Modèle CheXpert
try:
    models['chexpert'] = xrv.models.DenseNet(weights="densenet121-res224-chexpert")
    models['chexpert'].to(device)
    models['chexpert'].eval()
    model_info['chexpert'] = {
        'name': 'CheXpert',
        'pathologies': models['chexpert'].pathologies,
        'description': 'Spécialisé pour données CheXpert'
    }
    print("✅ CheXpert chargé / loaded")
except Exception as e:
    print(f"❌ Erreur CheXpert: {e}")

# 3. Modèle NIH
try:
    models['nih'] = xrv.models.DenseNet(weights="densenet121-res224-nih")
    models['nih'].to(device)
    models['nih'].eval()
    model_info['nih'] = {
        'name': 'NIH',
        'pathologies': models['nih'].pathologies,
        'description': 'Entraîné sur dataset NIH'
    }
    print("✅ NIH chargé / loaded")
except Exception as e:
    print(f"❌ Erreur NIH: {e}")

if not models:
    raise Exception("Aucun modèle n'a pu être chargé")

# Sélectionner le modèle principal
main_model_key = list(models.keys())[0]
main_model = models[main_model_key]

print(f"\n🎯 Modèles disponibles: {list(models.keys())}")
print(f"🏥 Modèle principal: {model_info[main_model_key]['name']}")
print(f"📊 Pathologies détectables: {len(main_model.pathologies)}")

## 📤 Chargement de votre Dataset / Upload Your Dataset

### 🎯 Méthodes de Chargement Faciles / Easy Upload Methods

Choisissez votre méthode préférée :

### Méthode 1 : Images Individuelles / Individual Images
Idéal pour 1-10 images

In [ ]:
# Chargement d'images individuelles / Upload individual images
print("📤 MÉTHODE 1: CHARGEMENT D'IMAGES INDIVIDUELLES")
print("📤 METHOD 1: INDIVIDUAL IMAGE UPLOAD")
print("-" * 60)
print("💡 Conseil: Cette méthode est idéale pour 1-10 images")
print("💡 Tip: This method is ideal for 1-10 images")
print("")
print("🔸 Formats supportés / Supported formats: .jpg, .jpeg, .png, .tiff")
print("🔸 Vous pouvez sélectionner plusieurs images en même temps")
print("🔸 You can select multiple images at once")
print("")
print("👆 Cliquez 'Choisir des fichiers' ci-dessous:")

# Interface de chargement
uploaded_files = files.upload()

individual_images = []
individual_filenames = []

if uploaded_files:
    print(f"\n📁 {len(uploaded_files)} fichier(s) chargé(s) / file(s) uploaded")
    
    for filename, file_content in uploaded_files.items():
        try:
            # Vérifier l'extension
            if not filename.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff', '.tif')):
                print(f"⚠️ {filename}: Format non supporté")
                continue
            
            # Charger l'image
            image = Image.open(io.BytesIO(file_content))
            
            # Convertir en niveaux de gris si nécessaire
            if image.mode != 'L':
                image = image.convert('L')
            
            # Convertir en array numpy
            img_array = np.array(image)
            
            individual_images.append(img_array)
            individual_filenames.append(filename)
            
            print(f"✅ {filename}: {img_array.shape} - Chargée avec succès")
            
        except Exception as e:
            print(f"❌ Erreur avec {filename}: {e}")
    
    print(f"\n🎉 {len(individual_images)} image(s) prête(s) pour l'analyse!")
    
    # Aperçu rapide des images chargées
    if len(individual_images) > 0:
        n_preview = min(4, len(individual_images))
        fig, axes = plt.subplots(1, n_preview, figsize=(4*n_preview, 4))
        
        if n_preview == 1:
            axes = [axes]
        
        for i in range(n_preview):
            axes[i].imshow(individual_images[i], cmap='gray')
            axes[i].set_title(f'{individual_filenames[i][:20]}...\n{individual_images[i].shape}')
            axes[i].axis('off')
        
        plt.suptitle('Aperçu des Images Chargées / Preview of Uploaded Images')
        plt.tight_layout()
        plt.show()

else:
    print("ℹ️ Aucune image chargée / No images uploaded")

### Méthode 2 : Archive ZIP / ZIP Archive
Idéal pour beaucoup d'images (10+)

In [ ]:
# Chargement d'archive ZIP / Upload ZIP archive
print("📦 MÉTHODE 2: CHARGEMENT D'ARCHIVE ZIP")
print("📦 METHOD 2: ZIP ARCHIVE UPLOAD")
print("-" * 60)
print("💡 Conseil: Cette méthode est idéale pour 10+ images")
print("💡 Tip: This method is ideal for 10+ images")
print("")
print("🔸 Créez un fichier .zip contenant vos images de radiographies")
print("🔸 Create a .zip file containing your X-ray images")
print("🔸 Les images peuvent être dans des sous-dossiers")
print("🔸 Images can be in subdirectories")
print("")
print("👆 Cliquez 'Choisir des fichiers' pour sélectionner votre fichier .zip:")

# Interface de chargement ZIP
uploaded_zip = files.upload()

zip_images = []
zip_filenames = []

if uploaded_zip:
    zip_filename = list(uploaded_zip.keys())[0]
    
    if zip_filename.lower().endswith('.zip'):
        print(f"📦 Extraction de {zip_filename}...")
        
        try:
            # Créer le fichier ZIP temporaire
            with open('temp_dataset.zip', 'wb') as f:
                f.write(uploaded_zip[zip_filename])
            
            # Extraire le ZIP
            with zipfile.ZipFile('temp_dataset.zip', 'r') as zip_ref:
                zip_ref.extractall('custom_dataset')
            
            # Trouver toutes les images dans l'extraction
            image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.tiff', '*.tif']
            image_files = []
            
            for ext in image_extensions:
                # Chercher récursivement
                image_files.extend(glob.glob(os.path.join('custom_dataset', '**', ext), recursive=True))
                image_files.extend(glob.glob(os.path.join('custom_dataset', '**', ext.upper()), recursive=True))
            
            print(f"🔍 {len(image_files)} image(s) trouvée(s) dans l'archive")
            
            # Charger toutes les images
            for img_path in tqdm(image_files, desc="Chargement des images"):
                try:
                    image = Image.open(img_path)
                    
                    # Convertir en niveaux de gris
                    if image.mode != 'L':
                        image = image.convert('L')
                    
                    img_array = np.array(image)
                    
                    zip_images.append(img_array)
                    zip_filenames.append(os.path.basename(img_path))
                    
                except Exception as e:
                    print(f"⚠️ Erreur avec {img_path}: {e}")
            
            print(f"\n🎉 {len(zip_images)} image(s) chargée(s) depuis l'archive!")
            
            # Statistiques du dataset
            if len(zip_images) > 0:
                sizes = [img.shape for img in zip_images]
                print(f"📊 Tailles d'images détectées: {set(sizes)}")
                
                # Aperçu d'un échantillon aléatoire
                n_sample = min(6, len(zip_images))
                sample_indices = np.random.choice(len(zip_images), n_sample, replace=False)
                
                fig, axes = plt.subplots(2, 3, figsize=(15, 10))
                axes = axes.flatten()
                
                for i, idx in enumerate(sample_indices):
                    axes[i].imshow(zip_images[idx], cmap='gray')
                    axes[i].set_title(f'{zip_filenames[idx][:20]}...\n{zip_images[idx].shape}')
                    axes[i].axis('off')
                
                plt.suptitle(f'Échantillon du Dataset (6/{len(zip_images)} images)')
                plt.tight_layout()
                plt.show()
            
            # Nettoyer le fichier temporaire
            os.remove('temp_dataset.zip')
            
        except Exception as e:
            print(f"❌ Erreur lors de l'extraction: {e}")
    
    else:
        print("❌ Le fichier chargé n'est pas une archive ZIP")

else:
    print("ℹ️ Aucune archive chargée / No archive uploaded")

### Méthode 3 : Dataset d'Exemple / Sample Dataset
Pour tester sans vos propres données

In [ ]:
# Création d'un dataset d'exemple pour test / Create sample dataset for testing
print("🧪 MÉTHODE 3: DATASET D'EXEMPLE POUR TEST")
print("🧪 METHOD 3: SAMPLE DATASET FOR TESTING")
print("-" * 60)
print("💡 Utilise cette option si vous n'avez pas encore vos propres images")
print("💡 Use this option if you don't have your own images yet")
print("")

def create_sample_dataset(n_images=8):
    """
    Créer un dataset d'exemple avec différents types de pathologies simulées
    Create sample dataset with different simulated pathology types
    """
    sample_images = []
    sample_filenames = []
    sample_descriptions = []
    
    # Types de cas simulés
    case_types = [
        ("normal", "Radiographie normale"),
        ("cardiomegaly", "Cardiomégalie simulée"),
        ("pneumonia", "Pneumonie simulée"),
        ("nodule", "Nodule pulmonaire simulé"),
        ("pneumothorax", "Pneumothorax simulé"),
        ("infiltration", "Infiltration pulmonaire simulée"),
        ("atelectasis", "Atélectasie simulée"),
        ("edema", "Œdème pulmonaire simulé")
    ]
    
    np.random.seed(42)  # Pour la reproductibilité
    
    for i in range(n_images):
        case_type, description = case_types[i % len(case_types)]
        
        # Image de base (thorax normal)
        img = np.random.rand(224, 224) * 0.3 + 0.4
        
        # Structures anatomiques de base
        # Poumons
        img[50:180, 30:100] *= 0.7   # Poumon gauche
        img[50:180, 124:194] *= 0.7  # Poumon droit
        
        # Cœur
        img[120:180, 90:134] *= 1.2
        
        # Ajouter pathologies spécifiques
        if case_type == "cardiomegaly":
            # Cœur élargi
            img[110:190, 80:144] *= 1.4
            
        elif case_type == "pneumonia":
            # Consolidation dans poumon droit
            img[80:140, 140:180] *= 1.6
            
        elif case_type == "nodule":
            # Nodule rond dans poumon gauche
            center_y, center_x = 100, 70
            y, x = np.ogrid[:224, :224]
            mask = (x - center_x)**2 + (y - center_y)**2 <= 8**2
            img[mask] *= 2.0
            
        elif case_type == "pneumothorax":
            # Zone hyperlucide (air) dans poumon droit
            img[60:120, 150:190] *= 0.3
            
        elif case_type == "infiltration":
            # Infiltrats diffus
            img[70:150, 40:90] *= 1.3    # Poumon gauche
            img[90:160, 130:170] *= 1.2  # Poumon droit
            
        elif case_type == "atelectasis":
            # Collapsus partiel poumon gauche
            img[80:130, 35:85] *= 1.7
            
        elif case_type == "edema":
            # Œdème diffus bilatéral
            img[60:170, 35:95] *= 1.4    # Gauche
            img[60:170, 129:189] *= 1.4  # Droit
            img[100:170, 85:139] *= 1.3  # Base cardiaque
        
        # Normaliser l'image
        img = np.clip(img, 0, 1)
        
        # Appliquer un léger flou pour un aspect plus réaliste
        img = cv2.GaussianBlur(img, (3, 3), 0)
        
        sample_images.append(img)
        sample_filenames.append(f"sample_{i+1:02d}_{case_type}.png")
        sample_descriptions.append(f"Cas {i+1}: {description}")
    
    return sample_images, sample_filenames, sample_descriptions

# Créer le dataset d'exemple
choice = input("Voulez-vous créer un dataset d'exemple ? (y/n): ").lower()

sample_images = []
sample_filenames = []
sample_descriptions = []

if choice in ['y', 'yes', 'oui', 'o']:
    print("\n🔄 Création du dataset d'exemple...")
    sample_images, sample_filenames, sample_descriptions = create_sample_dataset(8)
    
    print(f"✅ {len(sample_images)} images d'exemple créées!")
    
    # Affichage du dataset d'exemple
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for i, (img, filename, desc) in enumerate(zip(sample_images, sample_filenames, sample_descriptions)):
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(f'{filename}\n{desc}', fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle('Dataset d\'Exemple Créé / Sample Dataset Created', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
else:
    print("ℹ️ Dataset d'exemple non créé / Sample dataset not created")

## 📋 Consolidation du Dataset / Dataset Consolidation

Regroupons toutes les images chargées en un seul dataset :

In [ ]:
# Consolidation de toutes les images en un dataset unifié
print("📋 CONSOLIDATION DU DATASET")
print("📋 DATASET CONSOLIDATION")
print("=" * 50)

# Combiner toutes les sources d'images
all_images = []
all_filenames = []
all_sources = []

# Ajouter images individuelles
if 'individual_images' in locals() and individual_images:
    all_images.extend(individual_images)
    all_filenames.extend(individual_filenames)
    all_sources.extend(['Individual'] * len(individual_images))
    print(f"➕ {len(individual_images)} image(s) individuelles ajoutées")

# Ajouter images du ZIP
if 'zip_images' in locals() and zip_images:
    all_images.extend(zip_images)
    all_filenames.extend(zip_filenames)
    all_sources.extend(['ZIP Archive'] * len(zip_images))
    print(f"➕ {len(zip_images)} image(s) du ZIP ajoutées")

# Ajouter images d'exemple
if 'sample_images' in locals() and sample_images:
    all_images.extend(sample_images)
    all_filenames.extend(sample_filenames)
    all_sources.extend(['Sample'] * len(sample_images))
    print(f"➕ {len(sample_images)} image(s) d'exemple ajoutées")

# Vérifier si nous avons des images
if not all_images:
    print("❌ Aucune image disponible pour l'analyse!")
    print("💡 Veuillez charger des images en utilisant l'une des méthodes ci-dessus")
else:
    print(f"\n🎉 DATASET CONSOLIDÉ: {len(all_images)} image(s) au total")
    
    # Créer un DataFrame pour organiser les métadonnées
    dataset_info = pd.DataFrame({
        'ID': range(len(all_images)),
        'Filename': all_filenames,
        'Source': all_sources,
        'Shape': [img.shape for img in all_images],
        'Size_MB': [img.nbytes / 1024 / 1024 for img in all_images],
        'Min_Pixel': [img.min() for img in all_images],
        'Max_Pixel': [img.max() for img in all_images]
    })
    
    print("\n📊 INFORMATIONS DU DATASET:")
    print(dataset_info.to_string(index=False))
    
    # Statistiques générales
    print(f"\n📈 STATISTIQUES:")
    print(f"   • Nombre total d'images: {len(all_images)}")
    print(f"   • Tailles d'images uniques: {len(set([img.shape for img in all_images]))}")
    print(f"   • Taille totale du dataset: {sum(dataset_info['Size_MB']):.2f} MB")
    print(f"   • Sources: {', '.join(set(all_sources))}")
    
    # Affichage d'un résumé visuel
    if len(all_images) > 0:
        # Graphique des sources
        source_counts = pd.Series(all_sources).value_counts()
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Graphique en secteurs des sources
        axes[0].pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%')
        axes[0].set_title('Répartition par Source\nSource Distribution')
        
        # Histogramme des tailles d'images
        image_sizes = [img.shape[0] * img.shape[1] for img in all_images]
        axes[1].hist(image_sizes, bins=min(10, len(set(image_sizes))), alpha=0.7, edgecolor='black')
        axes[1].set_xlabel('Nombre de Pixels')
        axes[1].set_ylabel('Nombre d\'Images')
        axes[1].set_title('Distribution des Tailles\nSize Distribution')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("\n✅ Dataset consolidé et prêt pour l'analyse!")
    print("✅ Dataset consolidated and ready for analysis!")

## 🔧 Préparation en Lot / Batch Preprocessing

Préparons toutes les images pour l'analyse avec TorchXRayVision :

In [ ]:
def preprocess_dataset_batch(images, target_size=(224, 224)):
    """
    Préparation en lot de toutes les images du dataset
    Batch preprocessing of all images in the dataset
    """
    print("🔧 PRÉPARATION EN LOT DU DATASET")
    print("🔧 BATCH PREPROCESSING OF DATASET")
    print("=" * 50)
    
    processed_images = []
    processed_tensors = []
    preprocessing_info = []
    
    print(f"📏 Redimensionnement vers: {target_size}")
    print(f"🎯 Nombre d'images à traiter: {len(images)}")
    print("")
    
    for i, img in enumerate(tqdm(images, desc="Préparation des images")):
        try:
            # Copie de l'image originale
            img_work = img.copy()
            
            # Informations originales
            original_shape = img_work.shape
            original_dtype = img_work.dtype
            original_range = (img_work.min(), img_work.max())
            
            # Redimensionnement si nécessaire
            if img_work.shape != target_size:
                img_work = cv2.resize(img_work, target_size)
            
            # Normalisation des pixels entre 0 et 1
            if img_work.max() > 1:
                img_work = img_work.astype(np.float32) / 255.0
            else:
                img_work = img_work.astype(np.float32)
            
            # Normalisation Z-score pour TorchXRayVision
            mean_val = np.mean(img_work)
            std_val = np.std(img_work)
            
            if std_val > 0:
                img_normalized = (img_work - mean_val) / std_val
            else:
                img_normalized = img_work - mean_val
            
            # Conversion en tensor PyTorch
            img_tensor = torch.FloatTensor(img_normalized)
            img_tensor = img_tensor.unsqueeze(0).unsqueeze(0)  # [1, 1, H, W]
            img_tensor = img_tensor.to(device)
            
            # Stocker les résultats
            processed_images.append(img_work)
            processed_tensors.append(img_tensor)
            
            # Informations de préparation
            prep_info = {
                'original_shape': original_shape,
                'original_dtype': str(original_dtype),
                'original_range': original_range,
                'processed_shape': img_work.shape,
                'processed_range': (img_work.min(), img_work.max()),
                'normalized_range': (img_normalized.min(), img_normalized.max()),
                'mean': mean_val,
                'std': std_val,
                'tensor_shape': tuple(img_tensor.shape)
            }
            preprocessing_info.append(prep_info)
            
        except Exception as e:
            print(f"❌ Erreur avec l'image {i}: {e}")
            continue
    
    print(f"\n✅ {len(processed_images)}/{len(images)} images préparées avec succès")
    
    # Statistiques de préparation
    if preprocessing_info:
        prep_df = pd.DataFrame(preprocessing_info)
        
        print(f"\n📊 STATISTIQUES DE PRÉPARATION:")
        print(f"   • Forme finale: {target_size}")
        print(f"   • Range normalisé moyen: [{prep_df['normalized_range'].apply(lambda x: x[0]).mean():.3f}, {prep_df['normalized_range'].apply(lambda x: x[1]).mean():.3f}]")
        print(f"   • Moyenne des moyennes: {prep_df['mean'].mean():.3f}")
        print(f"   • Moyenne des écarts-types: {prep_df['std'].mean():.3f}")
        
        # Graphique des statistiques de normalisation
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # Distribution des moyennes
        axes[0].hist(prep_df['mean'], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0].set_xlabel('Moyenne des Pixels')
        axes[0].set_ylabel('Nombre d\'Images')
        axes[0].set_title('Distribution des Moyennes\nper Image')
        axes[0].grid(True, alpha=0.3)
        
        # Distribution des écarts-types
        axes[1].hist(prep_df['std'], bins=15, alpha=0.7, color='lightcoral', edgecolor='black')
        axes[1].set_xlabel('Écart-type des Pixels')
        axes[1].set_ylabel('Nombre d\'Images')
        axes[1].set_title('Distribution des Écarts-types\nper Image')
        axes[1].grid(True, alpha=0.3)
        
        # Corrélation moyenne vs écart-type
        axes[2].scatter(prep_df['mean'], prep_df['std'], alpha=0.7, s=60)
        axes[2].set_xlabel('Moyenne des Pixels')
        axes[2].set_ylabel('Écart-type des Pixels')
        axes[2].set_title('Corrélation Moyenne vs\nÉcart-type')
        axes[2].grid(True, alpha=0.3)
        
        plt.suptitle('Statistiques de Normalisation du Dataset', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    return processed_images, processed_tensors, preprocessing_info

# Appliquer le préprocessing si nous avons des images
if 'all_images' in locals() and all_images:
    processed_images, processed_tensors, prep_info = preprocess_dataset_batch(all_images)
    
    print("\n🎯 Dataset prêt pour l'analyse avec TorchXRayVision!")
    print("🎯 Dataset ready for TorchXRayVision analysis!")
else:
    print("❌ Aucune image à préparer")

## 🔍 Analyse en Lot / Batch Analysis

Analysons maintenant toutes les images avec les modèles TorchXRayVision :

In [ ]:
def analyze_dataset_batch(processed_tensors, filenames, models_dict, threshold=0.3):
    """
    Analyse en lot de tout le dataset avec tous les modèles disponibles
    Batch analysis of entire dataset with all available models
    """
    print("🔍 ANALYSE EN LOT DU DATASET")
    print("🔍 BATCH DATASET ANALYSIS")
    print("=" * 60)
    
    print(f"📊 Images à analyser: {len(processed_tensors)}")
    print(f"🤖 Modèles disponibles: {list(models_dict.keys())}")
    print(f"🎯 Seuil de détection: {threshold}")
    print("")
    
    # Stocker tous les résultats
    all_results = {}
    analysis_summary = []
    
    # Analyser avec chaque modèle
    for model_name, model in models_dict.items():
        print(f"🔄 Analyse avec le modèle {model_name}...")
        
        model_results = []
        pathologies = model.pathologies
        
        # Analyser chaque image
        for i, (tensor, filename) in enumerate(tqdm(zip(processed_tensors, filenames), 
                                                   desc=f"Analyse {model_name}",
                                                   total=len(processed_tensors))):
            try:
                with torch.no_grad():
                    outputs = model(tensor)
                    probabilities = torch.sigmoid(outputs).cpu().numpy().flatten()
                
                # Créer le résultat pour cette image
                image_result = {
                    'image_id': i,
                    'filename': filename,
                    'model': model_name
                }
                
                # Ajouter les probabilités pour chaque pathologie
                for pathology, prob in zip(pathologies, probabilities):
                    image_result[pathology] = prob
                    image_result[f'{pathology}_detected'] = prob > threshold
                
                # Compter les détections totales
                image_result['total_detections'] = sum(prob > threshold for prob in probabilities)
                image_result['max_probability'] = max(probabilities)
                image_result['avg_probability'] = np.mean(probabilities)
                
                model_results.append(image_result)
                
            except Exception as e:
                print(f"❌ Erreur avec {filename} sur {model_name}: {e}")
                continue
        
        all_results[model_name] = model_results
        
        # Résumé pour ce modèle
        if model_results:
            total_detections = sum(result['total_detections'] for result in model_results)
            avg_detections_per_image = total_detections / len(model_results)
            
            summary = {
                'model': model_name,
                'images_analyzed': len(model_results),
                'total_detections': total_detections,
                'avg_detections_per_image': avg_detections_per_image,
                'max_detections_single_image': max(result['total_detections'] for result in model_results),
                'images_with_detections': sum(1 for result in model_results if result['total_detections'] > 0)
            }
            analysis_summary.append(summary)
    
    print(f"\n✅ Analyse terminée pour {len(processed_tensors)} images")
    
    # Affichage du résumé
    if analysis_summary:
        summary_df = pd.DataFrame(analysis_summary)
        
        print("\n📊 RÉSUMÉ DE L'ANALYSE:")
        print(summary_df.to_string(index=False))
        
        # Graphiques de résumé
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        
        # 1. Détections totales par modèle
        axes[0, 0].bar(summary_df['model'], summary_df['total_detections'], 
                      color=['skyblue', 'lightcoral', 'lightgreen'][:len(summary_df)])
        axes[0, 0].set_title('Détections Totales par Modèle')
        axes[0, 0].set_ylabel('Nombre de Détections')
        axes[0, 0].tick_params(axis='x', rotation=45)
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. Moyenne de détections par image
        axes[0, 1].bar(summary_df['model'], summary_df['avg_detections_per_image'],
                      color=['orange', 'purple', 'brown'][:len(summary_df)])
        axes[0, 1].set_title('Moyenne Détections/Image')
        axes[0, 1].set_ylabel('Détections par Image')
        axes[0, 1].tick_params(axis='x', rotation=45)
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Images avec vs sans détections
        models_names = summary_df['model'].tolist()
        with_detections = summary_df['images_with_detections'].tolist()
        without_detections = [summary_df['images_analyzed'].iloc[i] - with_detections[i] 
                            for i in range(len(with_detections))]
        
        x = np.arange(len(models_names))
        width = 0.35
        
        axes[1, 0].bar(x, with_detections, width, label='Avec Détections', color='tomato')
        axes[1, 0].bar(x, without_detections, width, bottom=with_detections, 
                      label='Sans Détections', color='lightblue')
        axes[1, 0].set_title('Images avec/sans Détections')
        axes[1, 0].set_ylabel('Nombre d\'Images')
        axes[1, 0].set_xticks(x)
        axes[1, 0].set_xticklabels(models_names, rotation=45)
        axes[1, 0].legend()
        
        # 4. Maximum de détections sur une seule image
        axes[1, 1].bar(summary_df['model'], summary_df['max_detections_single_image'],
                      color=['gold', 'silver', 'bronze'][:len(summary_df)])
        axes[1, 1].set_title('Maximum Détections\n(Une Seule Image)')
        axes[1, 1].set_ylabel('Nombre de Détections')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.suptitle('RÉSUMÉ DE L\'ANALYSE DU DATASET', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    return all_results, analysis_summary

# Effectuer l'analyse si nous avons des tenseurs préparés
if 'processed_tensors' in locals() and processed_tensors:
    batch_results, batch_summary = analyze_dataset_batch(processed_tensors, all_filenames, models)
    
    print("\n🎉 Analyse en lot terminée avec succès!")
    print("🎉 Batch analysis completed successfully!")
else:
    print("❌ Aucun tensor préparé pour l'analyse")

## 📊 Analyse Détaillée des Résultats / Detailed Results Analysis

In [ ]:
def create_detailed_analysis(batch_results, filenames, threshold=0.3):
    """
    Analyse détaillée avec visualisations pour l'ensemble du dataset
    Detailed analysis with visualizations for the entire dataset
    """
    print("📊 ANALYSE DÉTAILLÉE DES RÉSULTATS")
    print("📊 DETAILED RESULTS ANALYSIS")
    print("=" * 60)
    
    if not batch_results:
        print("❌ Aucun résultat disponible pour l'analyse")
        return
    
    # Prendre le premier modèle pour l'analyse principale
    main_model = list(batch_results.keys())[0]
    main_results = batch_results[main_model]
    
    print(f"🎯 Analyse principale basée sur le modèle: {main_model}")
    print(f"📈 Nombre d'images analysées: {len(main_results)}")
    
    # 1. Analyse des pathologies les plus fréquentes
    pathologies = [col for col in main_results[0].keys() 
                  if col not in ['image_id', 'filename', 'model', 'total_detections', 
                               'max_probability', 'avg_probability'] 
                  and not col.endswith('_detected')]
    
    # Compter les détections par pathologie
    pathology_counts = {}
    pathology_avg_probs = {}
    
    for pathology in pathologies:
        detections = sum(1 for result in main_results if result.get(f'{pathology}_detected', False))
        avg_prob = np.mean([result.get(pathology, 0) for result in main_results])
        
        pathology_counts[pathology] = detections
        pathology_avg_probs[pathology] = avg_prob
    
    # Trier par fréquence
    sorted_pathologies = sorted(pathology_counts.items(), key=lambda x: x[1], reverse=True)
    
    print(f"\n🏥 TOP 10 PATHOLOGIES DÉTECTÉES:")
    for i, (pathology, count) in enumerate(sorted_pathologies[:10]):
        percentage = (count / len(main_results)) * 100
        avg_prob = pathology_avg_probs[pathology]
        print(f"   {i+1:2d}. {pathology:25s}: {count:3d}/{len(main_results)} ({percentage:5.1f}%) - Prob moy: {avg_prob:.3f}")
    
    # 2. Analyse des images les plus problématiques
    print(f"\n🚨 IMAGES AVEC LE PLUS DE PATHOLOGIES:")
    sorted_images = sorted(main_results, key=lambda x: x['total_detections'], reverse=True)
    
    for i, result in enumerate(sorted_images[:5]):
        detected_paths = [path for path in pathologies 
                         if result.get(f'{path}_detected', False)]
        print(f"   {i+1}. {result['filename']:30s}: {result['total_detections']} détections")
        if detected_paths:
            print(f"      → {', '.join(detected_paths[:3])}{'...' if len(detected_paths) > 3 else ''}")
    
    # 3. Visualisations complètes
    create_comprehensive_visualizations(batch_results, pathologies, filenames, threshold)
    
    # 4. Comparaison inter-modèles si disponible
    if len(batch_results) > 1:
        create_model_comparison_analysis(batch_results, pathologies)
    
    # 5. Matrice de corrélation des pathologies
    create_pathology_correlation_matrix(main_results, pathologies)
    
    return sorted_pathologies, sorted_images

def create_comprehensive_visualizations(batch_results, pathologies, filenames, threshold):
    """
    Créer des visualisations complètes pour l'analyse du dataset
    Create comprehensive visualizations for dataset analysis
    """
    main_model = list(batch_results.keys())[0]
    results = batch_results[main_model]
    
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(4, 3, height_ratios=[1, 1, 1, 1], hspace=0.3, wspace=0.3)
    
    # 1. Heatmap des détections par image et pathologie
    ax1 = fig.add_subplot(gs[0, :])
    
    # Créer matrice de détection
    top_pathologies = sorted(pathologies, 
                           key=lambda p: sum(1 for r in results if r.get(f'{p}_detected', False)), 
                           reverse=True)[:15]  # Top 15 pour la lisibilité
    
    detection_matrix = []
    image_labels = []
    
    for result in results:
        row = [1 if result.get(f'{path}_detected', False) else 0 for path in top_pathologies]
        detection_matrix.append(row)
        image_labels.append(result['filename'][:15] + '...' if len(result['filename']) > 15 else result['filename'])
    
    detection_matrix = np.array(detection_matrix).T  # Transpose pour avoir pathologies en lignes
    
    sns.heatmap(detection_matrix, 
                xticklabels=image_labels,
                yticklabels=[p[:15] for p in top_pathologies],
                cmap='Reds', cbar_kws={'label': 'Détection (0=Non, 1=Oui)'},
                ax=ax1)
    ax1.set_title(f'Matrice de Détection - {len(results)} Images vs Top 15 Pathologies', 
                 fontsize=14, fontweight='bold')
    ax1.set_xlabel('Images du Dataset')
    ax1.set_ylabel('Pathologies')
    
    # Rotation des labels pour lisibilité
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    
    # 2. Distribution des détections par image
    ax2 = fig.add_subplot(gs[1, 0])
    detections_per_image = [result['total_detections'] for result in results]
    ax2.hist(detections_per_image, bins=range(max(detections_per_image)+2), 
            alpha=0.7, color='skyblue', edgecolor='black')
    ax2.set_xlabel('Nombre de Détections')
    ax2.set_ylabel('Nombre d\'Images')
    ax2.set_title('Distribution des\nDétections par Image')
    ax2.grid(True, alpha=0.3)
    
    # 3. Top pathologies (barres)
    ax3 = fig.add_subplot(gs[1, 1])
    top_10_paths = top_pathologies[:10]
    counts = [sum(1 for r in results if r.get(f'{path}_detected', False)) for path in top_10_paths]
    
    bars = ax3.barh(range(len(top_10_paths)), counts, color='lightcoral', alpha=0.7)
    ax3.set_yticks(range(len(top_10_paths)))
    ax3.set_yticklabels([p[:12] for p in top_10_paths])
    ax3.set_xlabel('Nombre de Détections')
    ax3.set_title('Top 10 Pathologies\nles Plus Fréquentes')
    ax3.grid(True, alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for i, (bar, count) in enumerate(zip(bars, counts)):
        width = bar.get_width()
        ax3.text(width + 0.1, bar.get_y() + bar.get_height()/2, 
                f'{count}', ha='left', va='center', fontweight='bold')
    
    # 4. Distribution des probabilités moyennes
    ax4 = fig.add_subplot(gs[1, 2])
    avg_probs = [result['avg_probability'] for result in results]
    ax4.hist(avg_probs, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
    ax4.axvline(x=threshold, color='red', linestyle='--', linewidth=2, label=f'Seuil ({threshold})')
    ax4.set_xlabel('Probabilité Moyenne')
    ax4.set_ylabel('Nombre d\'Images')
    ax4.set_title('Distribution des\nProbabilités Moyennes')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Corrélation entre détections et probabilités max
    ax5 = fig.add_subplot(gs[2, 0])
    max_probs = [result['max_probability'] for result in results]
    ax5.scatter(detections_per_image, max_probs, alpha=0.6, s=50)
    ax5.set_xlabel('Nombre de Détections')
    ax5.set_ylabel('Probabilité Maximale')
    ax5.set_title('Détections vs\nProbabilité Max')
    ax5.grid(True, alpha=0.3)
    
    # 6. Images par catégorie de sévérité
    ax6 = fig.add_subplot(gs[2, 1])
    
    # Catégoriser les images par sévérité
    normal_images = sum(1 for r in results if r['total_detections'] == 0)
    mild_images = sum(1 for r in results if 1 <= r['total_detections'] <= 2)
    moderate_images = sum(1 for r in results if 3 <= r['total_detections'] <= 5)
    severe_images = sum(1 for r in results if r['total_detections'] > 5)
    
    categories = ['Normal\n(0)', 'Léger\n(1-2)', 'Modéré\n(3-5)', 'Sévère\n(6+)']
    counts_sev = [normal_images, mild_images, moderate_images, severe_images]
    colors_sev = ['green', 'yellow', 'orange', 'red']
    
    wedges, texts, autotexts = ax6.pie(counts_sev, labels=categories, colors=colors_sev,
                                      autopct='%1.1f%%', startangle=90)
    ax6.set_title('Répartition par\nNiveau de Sévérité')
    
    # 7. Timeline/ordre des images
    ax7 = fig.add_subplot(gs[2, 2])
    image_indices = range(len(results))
    ax7.plot(image_indices, detections_per_image, 'bo-', alpha=0.7, markersize=4)
    ax7.set_xlabel('Index de l\'Image')
    ax7.set_ylabel('Nombre de Détections')
    ax7.set_title('Évolution des Détections\nSelon l\'Ordre')
    ax7.grid(True, alpha=0.3)
    
    # 8. Comparaison probabilités vs détections (scatter large)
    ax8 = fig.add_subplot(gs[3, :])
    
    # Pour chaque pathologie, afficher prob moyenne vs fréquence de détection
    path_freq = []
    path_avg_prob = []
    path_names = []
    
    for pathology in top_pathologies[:20]:  # Top 20
        freq = sum(1 for r in results if r.get(f'{pathology}_detected', False))
        avg_prob = np.mean([r.get(pathology, 0) for r in results])
        
        path_freq.append(freq)
        path_avg_prob.append(avg_prob)
        path_names.append(pathology)
    
    scatter = ax8.scatter(path_freq, path_avg_prob, s=100, alpha=0.7, c=range(len(path_freq)), cmap='viridis')
    
    # Ajouter les noms des pathologies
    for i, (freq, prob, name) in enumerate(zip(path_freq, path_avg_prob, path_names)):
        ax8.annotate(name[:10], (freq, prob), xytext=(5, 5), textcoords='offset points', 
                    fontsize=8, alpha=0.8)
    
    ax8.set_xlabel('Fréquence de Détection (nombre d\'images)')
    ax8.set_ylabel('Probabilité Moyenne')
    ax8.set_title('Analyse Pathologie: Fréquence vs Probabilité Moyenne', fontweight='bold')
    ax8.grid(True, alpha=0.3)
    
    plt.suptitle(f'ANALYSE COMPLÈTE DU DATASET - {len(results)} Images', 
                fontsize=18, fontweight='bold')
    plt.show()

def create_pathology_correlation_matrix(results, pathologies):
    """
    Créer une matrice de corrélation entre pathologies
    Create correlation matrix between pathologies
    """
    print("\n🔗 ANALYSE DES CORRÉLATIONS ENTRE PATHOLOGIES")
    
    # Créer matrice de probabilités
    prob_matrix = []
    for pathology in pathologies:
        probs = [result.get(pathology, 0) for result in results]
        prob_matrix.append(probs)
    
    prob_matrix = np.array(prob_matrix)
    correlation_matrix = np.corrcoef(prob_matrix)
    
    # Visualiser seulement les corrélations les plus significatives
    plt.figure(figsize=(12, 10))
    
    # Masquer la diagonale et les corrélations faibles
    mask = np.zeros_like(correlation_matrix, dtype=bool)
    mask[np.triu_indices_from(mask)] = True
    mask[np.abs(correlation_matrix) < 0.3] = True
    
    sns.heatmap(correlation_matrix, 
                mask=mask,
                annot=True, 
                fmt='.2f', 
                cmap='RdBu_r',
                vmin=-1, vmax=1,
                xticklabels=[p[:12] for p in pathologies],
                yticklabels=[p[:12] for p in pathologies],
                cbar_kws={'label': 'Coefficient de Corrélation'})
    
    plt.title('Matrice de Corrélation entre Pathologies\n(Seulement corrélations |r| > 0.3)', 
             fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Trouver les corrélations les plus fortes
    strong_correlations = []
    for i in range(len(pathologies)):
        for j in range(i+1, len(pathologies)):
            corr = correlation_matrix[i, j]
            if abs(corr) > 0.5:  # Corrélations fortes
                strong_correlations.append((pathologies[i], pathologies[j], corr))
    
    if strong_correlations:
        print("\n🔗 CORRÉLATIONS FORTES DÉTECTÉES (|r| > 0.5):")
        strong_correlations.sort(key=lambda x: abs(x[2]), reverse=True)
        for path1, path2, corr in strong_correlations[:10]:
            direction = "positive" if corr > 0 else "négative"
            print(f"   • {path1} ↔ {path2}: r = {corr:.3f} ({direction})")

# Effectuer l'analyse détaillée si nous avons des résultats
if 'batch_results' in locals() and batch_results:
    top_pathologies, problematic_images = create_detailed_analysis(batch_results, all_filenames)
else:
    print("❌ Aucun résultat disponible pour l'analyse détaillée")

## 💾 Export des Résultats / Export Results

Sauvegardons et exportons tous les résultats pour usage ultérieur :

In [ ]:
def export_dataset_results(batch_results, filenames, processed_images, analysis_summary):
    """
    Exporter tous les résultats dans différents formats
    Export all results in different formats
    """
    print("💾 EXPORT DES RÉSULTATS")
    print("💾 EXPORTING RESULTS")
    print("=" * 50)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Export en CSV (pour Excel, analyse statistique)
    print("📊 Export CSV...")
    
    for model_name, results in batch_results.items():
        df = pd.DataFrame(results)
        csv_filename = f'results/dataset_analysis_{model_name}_{timestamp}.csv'
        df.to_csv(csv_filename, index=False)
        print(f"   ✅ {csv_filename}")
    
    # 2. Export du résumé exécutif
    print("📋 Export résumé exécutif...")
    
    summary_df = pd.DataFrame(analysis_summary)
    summary_filename = f'results/dataset_summary_{timestamp}.csv'
    summary_df.to_csv(summary_filename, index=False)
    print(f"   ✅ {summary_filename}")
    
    # 3. Export JSON complet (pour intégration avec d'autres outils)
    print("🗂️ Export JSON...")
    
    json_data = {
        'metadata': {
            'timestamp': timestamp,
            'total_images': len(filenames),
            'models_used': list(batch_results.keys()),
            'analysis_date': datetime.now().isoformat()
        },
        'results': batch_results,
        'summary': analysis_summary,
        'filenames': filenames
    }
    
    json_filename = f'results/complete_analysis_{timestamp}.json'
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False, default=str)
    print(f"   ✅ {json_filename}")
    
    # 4. Rapport textuel détaillé
    print("📝 Génération rapport textuel...")
    
    report_filename = f'results/detailed_report_{timestamp}.txt'
    generate_detailed_text_report(batch_results, analysis_summary, report_filename, timestamp)
    print(f"   ✅ {report_filename}")
    
    # 5. Package de téléchargement
    print("📦 Création package de téléchargement...")
    
    # Zipper tous les résultats
    zip_filename = f'dataset_analysis_complete_{timestamp}.zip'
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        # Ajouter tous les fichiers du dossier results
        for filename in glob.glob('results/*'):
            if timestamp in filename:
                zipf.write(filename, os.path.basename(filename))
    
    print(f"   ✅ {zip_filename}")
    
    # 6. Interface de téléchargement Colab
    print("\n📤 TÉLÉCHARGEMENT DES RÉSULTATS:")
    print("Choisissez quels fichiers télécharger:")
    
    download_options = {
        '1': ('Package complet (ZIP)', zip_filename),
        '2': ('Résumé exécutif (CSV)', summary_filename),
        '3': ('Rapport détaillé (TXT)', report_filename),
        '4': ('Données complètes (JSON)', json_filename)
    }
    
    for key, (desc, filename) in download_options.items():
        print(f"   {key}. {desc}")
    
    # Interface de sélection
    choice = input("\nEntrez le numéro de votre choix (ou 'all' pour tout télécharger): ")
    
    if choice.lower() == 'all':
        for desc, filename in download_options.values():
            try:
                files.download(filename)
                print(f"✅ Téléchargé: {filename}")
            except Exception as e:
                print(f"❌ Erreur téléchargement {filename}: {e}")
    elif choice in download_options:
        desc, filename = download_options[choice]
        try:
            files.download(filename)
            print(f"✅ Téléchargé: {filename}")
        except Exception as e:
            print(f"❌ Erreur téléchargement: {e}")
    else:
        print("ℹ️ Aucun téléchargement sélectionné")
    
    return {
        'zip_file': zip_filename,
        'summary_file': summary_filename,
        'report_file': report_filename,
        'json_file': json_filename
    }

def generate_detailed_text_report(batch_results, analysis_summary, filename, timestamp):
    """
    Générer un rapport textuel détaillé
    Generate detailed text report
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("=" * 80 + "\n")
        f.write("RAPPORT D'ANALYSE DE DATASET PERSONNALISÉ TORCHXRAYVISION\n")
        f.write("CUSTOM DATASET ANALYSIS REPORT - TORCHXRAYVISION\n")
        f.write("=" * 80 + "\n")
        f.write(f"Date d'analyse: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write("\n")
        
        # Résumé exécutif
        f.write("RÉSUMÉ EXÉCUTIF\n")
        f.write("-" * 50 + "\n")
        
        total_images = len(batch_results[list(batch_results.keys())[0]])
        f.write(f"Nombre total d'images analysées: {total_images}\n")
        f.write(f"Modèles utilisés: {', '.join(batch_results.keys())}\n")
        
        for summary in analysis_summary:
            f.write(f"\nModèle {summary['model']}:\n")
            f.write(f"  - Images avec détections: {summary['images_with_detections']}/{summary['images_analyzed']}\n")
            f.write(f"  - Détections moyennes par image: {summary['avg_detections_per_image']:.2f}\n")
            f.write(f"  - Maximum détections (une image): {summary['max_detections_single_image']}\n")
        
        # Détails par modèle
        for model_name, results in batch_results.items():
            f.write(f"\n\n{'='*60}\n")
            f.write(f"ANALYSE DÉTAILLÉE - MODÈLE {model_name}\n")
            f.write(f"{'='*60}\n")
            
            # Top pathologies
            pathologies = [col for col in results[0].keys() 
                          if col not in ['image_id', 'filename', 'model', 'total_detections', 
                                       'max_probability', 'avg_probability'] 
                          and not col.endswith('_detected')]
            
            pathology_counts = {}
            for pathology in pathologies:
                count = sum(1 for result in results if result.get(f'{pathology}_detected', False))
                pathology_counts[pathology] = count
            
            sorted_paths = sorted(pathology_counts.items(), key=lambda x: x[1], reverse=True)
            
            f.write("\nTOP 15 PATHOLOGIES DÉTECTÉES:\n")
            for i, (pathology, count) in enumerate(sorted_paths[:15]):
                percentage = (count / len(results)) * 100
                f.write(f"  {i+1:2d}. {pathology:30s}: {count:3d} ({percentage:5.1f}%)\n")
            
            # Images les plus problématiques
            f.write("\nIMAGES LES PLUS PROBLÉMATIQUES:\n")
            sorted_images = sorted(results, key=lambda x: x['total_detections'], reverse=True)
            
            for i, result in enumerate(sorted_images[:10]):
                f.write(f"  {i+1:2d}. {result['filename']:40s}: {result['total_detections']} détections\n")
        
        # Recommandations
        f.write(f"\n\n{'='*60}\n")
        f.write("RECOMMANDATIONS ET ACTIONS SUGGÉRÉES\n")
        f.write(f"{'='*60}\n")
        
        main_results = batch_results[list(batch_results.keys())[0]]
        high_detection_images = [r for r in main_results if r['total_detections'] > 5]
        
        if high_detection_images:
            f.write(f"\nATTENTION: {len(high_detection_images)} image(s) avec 6+ pathologies détectées.\n")
            f.write("Recommandation: Révision clinique prioritaire.\n")
        
        normal_images = [r for r in main_results if r['total_detections'] == 0]
        f.write(f"\nImages normales: {len(normal_images)}/{len(main_results)} ({len(normal_images)/len(main_results)*100:.1f}%)\n")
        
        f.write("\n" + "=" * 80 + "\n")
        f.write("Fin du rapport\n")
        f.write("=" * 80 + "\n")

# Effectuer l'export si nous avons des résultats
if 'batch_results' in locals() and batch_results:
    export_files = export_dataset_results(batch_results, all_filenames, processed_images, batch_summary)
    
    print("\n🎉 EXPORT TERMINÉ AVEC SUCCÈS!")
    print("🎉 EXPORT COMPLETED SUCCESSFULLY!")
    print("\n📋 Fichiers générés:")
    for desc, filename in export_files.items():
        print(f"   • {desc}: {filename}")
else:
    print("❌ Aucun résultat à exporter")

## 🎉 Résumé du Tutorial / Tutorial Summary

### Ce que vous avez accompli / What you accomplished:

In [ ]:
# Génération du résumé final du tutorial
print("🎉" * 50)
print("TUTORIAL 6 TERMINÉ AVEC SUCCÈS!")
print("TUTORIAL 6 COMPLETED SUCCESSFULLY!")
print("🎉" * 50)

print("\n📊 RÉSUMÉ DE VOS ACCOMPLISSEMENTS:")
print("📊 SUMMARY OF YOUR ACHIEVEMENTS:")
print("=" * 70)

if 'all_images' in locals() and all_images:
    print(f"\n✅ CHARGEMENT DE DONNÉES:")
    print(f"   • {len(all_images)} image(s) chargée(s) avec succès")
    
    # Détails par source
    source_summary = {}
    if 'all_sources' in locals():
        for source in set(all_sources):
            count = all_sources.count(source)
            source_summary[source] = count
        
        for source, count in source_summary.items():
            print(f"   • {source}: {count} image(s)")

if 'processed_images' in locals() and processed_images:
    print(f"\n✅ PRÉPARATION DES DONNÉES:")
    print(f"   • {len(processed_images)} image(s) préparée(s) pour l'analyse")
    print(f"   • Redimensionnement vers 224x224 pixels")
    print(f"   • Normalisation Z-score appliquée")
    print(f"   • Conversion en tenseurs PyTorch réussie")

if 'models' in locals() and models:
    print(f"\n✅ MODÈLES UTILISÉS:")
    for model_name, model_data in model_info.items():
        if model_name in models:
            print(f"   • {model_data['name']}: {len(model_data['pathologies'])} pathologies")

if 'batch_results' in locals() and batch_results:
    print(f"\n✅ ANALYSE RÉALISÉE:")
    
    main_model_key = list(batch_results.keys())[0]
    main_results = batch_results[main_model_key]
    
    total_detections = sum(result['total_detections'] for result in main_results)
    images_with_findings = sum(1 for result in main_results if result['total_detections'] > 0)
    max_detections = max(result['total_detections'] for result in main_results)
    
    print(f"   • {len(main_results)} image(s) analysée(s) avec IA")
    print(f"   • {total_detections} pathologies détectées au total")
    print(f"   • {images_with_findings}/{len(main_results)} image(s) avec anomalies")
    print(f"   • Maximum {max_detections} pathologies sur une image")

if 'export_files' in locals() and export_files:
    print(f"\n✅ RÉSULTATS EXPORTÉS:")
    print(f"   • Rapport CSV pour analyse statistique")
    print(f"   • Fichier JSON pour intégration logicielle")
    print(f"   • Rapport textuel détaillé")
    print(f"   • Package ZIP complet")

print(f"\n🎓 COMPÉTENCES ACQUISES:")
print(f"   🔹 Chargement facile de datasets personnalisés")
print(f"   🔹 Préparation automatique d'images médicales")
print(f"   🔹 Analyse en lot avec TorchXRayVision")
print(f"   🔹 Comparaison multi-modèles")
print(f"   🔹 Visualisation et interprétation des résultats")
print(f"   🔹 Export pour usage clinique et recherche")

print(f"\n🚀 APPLICATIONS PRATIQUES:")
print(f"   🏥 Analyse de cas cliniques personnels")
print(f"   📊 Études de recherche médicale")
print(f"   📚 Projets étudiants en médecine")
print(f"   🔬 Validation d'IA sur populations locales")
print(f"   📈 Audit qualité radiologique")

print(f"\n💡 PROCHAINES ÉTAPES SUGGÉRÉES:")
print(f"   📖 Révision des tutoriels 1-5 pour approfondir")
print(f"   🔄 Test avec vos propres données médicales")
print(f"   📊 Analyse statistique avancée des résultats")
print(f"   🤝 Collaboration avec radiologues pour validation")
print(f"   📝 Rédaction d'articles de recherche")

print(f"\n⚠️ RAPPELS IMPORTANTS:")
print(f"   🔐 Respecter la confidentialité des données patients")
print(f"   🏥 Toujours valider avec expertise médicale")
print(f"   📋 L'IA assiste mais ne remplace pas le médecin")
print(f"   📊 Documenter méthodes et limitations")

print(f"\n" + "="*70)
print(f"🙏 FÉLICITATIONS! Vous maîtrisez maintenant l'intégration")
print(f"   de datasets personnalisés avec TorchXRayVision!")
print(f"")
print(f"🌟 CONGRATULATIONS! You now master custom dataset")
print(f"   integration with TorchXRayVision!")
print(f"="*70)

# Suggestion pour continuer
print(f"\n🔄 POUR CONTINUER VOTRE APPRENTISSAGE:")
print(f"   • Testez avec différents types d'images")
print(f"   • Comparez les modèles sur vos données")
print(f"   • Intégrez dans vos workflows de recherche")
print(f"   • Partagez vos découvertes avec la communauté")

print(f"\n🎯 Vous êtes maintenant prêt(e) pour des applications")
print(f"   concrètes en recherche médicale et pratique clinique!")